In [3]:
from __future__ import print_function

import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from tensorflow.python.keras import layers
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras import backend as K

from data import load_train_data, load_test_data

K.set_image_data_format('channels_last')# TF dimension ordering in this code

img_rows = 96
img_cols = 96

smooth = 1.


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return dice_coef(y_true, y_pred)


def get_unet():
    inputs = Input((img_rows, img_cols, 1))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer='adam', loss=dice_coef_loss, metrics=[dice_coef])
    model.save('model.h5')
    return model


def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)

    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p


def train_and_predict():
    print('-'*30)
    print('Loading and preprocessing train data...')
    print('-'*30)
    imgs_train, imgs_mask_train = load_train_data()

    imgs_train = preprocess(imgs_train)
    imgs_mask_train = preprocess(imgs_mask_train)

    imgs_train = imgs_train.astype('float32')
    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization

    imgs_train -= mean
    imgs_train /= std

    imgs_mask_train = imgs_mask_train.astype('float32')
    imgs_mask_train /= 255.  # scale masks to [0, 1]

    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    model = get_unet()
    model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    model.fit(imgs_train, imgs_mask_train, batch_size=32, epochs=10, verbose=1, shuffle=True,
              validation_split=0.2,
              callbacks=[model_checkpoint])

    print('-'*30)
    print('Loading and preprocessing test data...')
    print('-'*30)
    imgs_test, imgs_id_test = load_test_data()
    imgs_test = preprocess(imgs_test)

    imgs_test = imgs_test.astype('float32')
    imgs_test -= mean
    imgs_test /= std

    print('-'*30)
    print('Loading saved weights...')
    print('-'*30)
    model.load_weights('weights.h5')

    print('-'*30)
    print('Predicting masks on test data...')
    print('-'*30)
    imgs_mask_test = model.predict(imgs_test, verbose=1)
    np.save('imgs_mask_test.npy', imgs_mask_test)

    print('-' * 30)
    print('Saving predicted masks to files...')
    print('-' * 30)
    pred_dir = 'preds'
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dir)
    for image, image_id in zip(imgs_mask_test, imgs_id_test):
        image = (image[:, :, 0] * 255.).astype(np.uint8)
        imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)

if __name__ == '__main__':
    train_and_predict()


------------------------------
Loading and preprocessing train data...
------------------------------
------------------------------
Creating and compiling model...
------------------------------
------------------------------
Fitting model...
------------------------------
Epoch 1/10
141/141 [==============================] - 927s 7s/step - loss: 0.0017 - dice_coef: 0.0017 - val_loss: 6.5605e-04 - val_dice_coef: 6.6891e-04
Epoch 2/10
141/141 [==============================] - 704s 5s/step - loss: 2.8595e-04 - dice_coef: 2.8599e-04 - val_loss: 6.5605e-04 - val_dice_coef: 6.6891e-04
Epoch 3/10
141/141 [==============================] - 690s 5s/step - loss: 2.8754e-04 - dice_coef: 2.8763e-04 - val_loss: 6.5605e-04 - val_dice_coef: 6.6891e-04
Epoch 4/10
141/141 [==============================] - 691s 5s/step - loss: 2.8509e-04 - dice_coef: 2.8529e-04 - val_loss: 6.5605e-04 - val_dice_coef: 6.6891e-04
Epoch 5/10
141/141 [==============================] - 684s 5s/step - loss: 2.8278e-04 - d

<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\10_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\100_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1000_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1001_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1002_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150

<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1103_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1104_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1105_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1106_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1107_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1108_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a

<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1195_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1196_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1197_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1198_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1199_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\12_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>

<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1296_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1297_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1298_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\1299_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\13_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\130_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:

...

<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4150_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4151_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4152_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4153_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4154_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4155_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a

<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4238_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4239_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\424_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4240_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4241_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4242_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5

<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4319_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\432_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4320_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4321_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4322_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4323_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5

<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4403_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4404_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4405_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4406_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4407_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4408_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a

<ipython-input-3-1da77db451a5>:150: UserWarning: preds\448_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4480_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4481_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4482_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4483_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5>:150: UserWarning: preds\4484_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-3-1da77db451a5